In [146]:
library(tidyverse)

In [147]:
# Make sure all columns of a tibble are being shown
options(repr.matrix.max.rows = Inf, repr.matrix.max.cols = Inf)

In [148]:
grant_data <- read_delim("3_BiomedGrantData_20230517.csv", delim = ";", col_names=TRUE)

Rows: 7609 Columns: 53
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr   (7): State, FundingInstrumentLevel1, ResearchInstitution, InstituteCou...
dbl  (41): GrantNumber, CallDecisionYear, GrantDurationDays, GrantDurationYe...
date  (5): EffectiveGrantStartDate, EffectiveGrantEndDate, DateFirstAcademic...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [149]:
dim(grant_data)
head(grant_data)

[1] 7609   53

GrantNumber,CallDecisionYear,EffectiveGrantStartDate,GrantDurationDays,GrantDurationYears,EffectiveGrantEndDate,State,FundingInstrumentLevel1,AmountGrantedAllSets,AmountPerYear,AmountPerEmployee,AmountPerEmployeeAndYear,ResearchInstitution,InstituteCountry,MainDiscipline_Level1,MainDiscipline_Level2,MainDiscipline,NrEmployees,NrEmployeesPerYear,NrCollaborations,NrCollaborationsPerYear,NrAcademicEvents,NrAcademicEventsPerYear,DateFirstAcademicEvent,NrAwards,NrAwardsPerYear,YearFirstAward,NrDatasets,NrDatasetsPerYear,DateFirstDataset,NrKnowledgeTransfer,NrKnowledgeTransferPerYear,DateFirstKnowledgeTransfer,NrCommunication,NrCommunicationPerYear,YearFirstCommunication,NrTotalPublications,NrTotalPublicationsPerYear,YearFirstPublication,NrUncitablePublications,NrUncitablePublicationsPerYear,NrUses,NrUsesPerYear,YearFirstUse,NrTotalOutputs,NrTotalOutputsPerYear,NrTotalUncitableOutputs,NrTotalUncitableOutputsPerYear,NrCitablePublications,NrCitablePublicationsPerYear,NrCitationsTotal,NrCitationsPerPublication,YearFirstCitablePublication
<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
142628,2012,2013-09-01,1490,4.0821918,2017-09-30,Completed,Careers,578812,141789.52,192937.33,47263.17,University of Lausanne - LA,Switzerland,Biology and Medicine,General Biology,Ecology,3,0.7348993,2,0.4899329,4,0.9798658,2015-03-10,1,0.2449664,2013,0,0,NA,0,0,NA,0,0.0000000,NA,8,1.9597315,2014,1,0.2449664,1,0.2449664,2015,14,3.429530,7,1.714765,7,1.7147651,293,41.85714,2014
142615,2012,2013-04-01,1095,3.0000000,2016-03-31,Completed,Careers,534863,178287.67,267431.50,89143.83,ETH Zurich - ETHZ,Switzerland,Biology and Medicine,Basic Biological Research,"Cellular Biology, Cytology",2,0.6666667,9,3.0000000,16,5.3333333,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,1,0.3333333,2013,8,2.6666667,NA,2,0.6666667,0,0.0000000,NA,25,8.333333,19,6.333333,6,2.0000000,NA,NA,NA
144100,2012,2013-09-01,1641,4.4958904,2018-02-28,Completed,Projects,291191,64768.26,72797.75,16192.07,University of Berne - BE,Switzerland,Biology and Medicine,Preventive Medicine (Epidemiology/Early Diagnosis/Prevention),"Mental Disorders, Psychosomatic Diseases",4,0.8897014,6,1.3345521,43,9.5642901,2014-03-01,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,4,0.8897014,NA,1,0.2224254,0,0.0000000,NA,47,10.453991,44,9.786715,3,0.6672761,145,48.33333,2015
147291,2012,2013-02-01,364,0.9972603,2014-01-31,Completed,Careers,60400,60565.93,0.00,0.00,Institution abroad - United States of America,United States of America,Biology and Medicine,Basic Biological Research,Biochemistry,0,0.0000000,1,1.0027473,1,1.0027473,2013-05-06,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,0,0.0000000,NA,0,0.0000000,0,0.0000000,NA,1,1.002747,1,1.002747,0,0.0000000,NA,NA,NA
142408,2012,2013-01-01,545,1.4931507,2014-06-30,Completed,Careers,584267,391298.08,584267.00,391298.08,University of Zurich - ZH,Switzerland,Biology and Medicine,Basic Biological Research,Experimental Microbiology,1,0.6697248,4,2.6788991,4,2.6788991,2013-05-07,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,2,1.3394495,2013,1,0.6697248,0,0.0000000,NA,6,4.018349,5,3.348624,1,0.6697248,NA,NA,2013
143733,2012,2013-01-01,1276,3.4958904,2016-06-30,Completed,Projects,500000,143025.08,125000.00,35756.27,University of Geneva - GE,Switzerland,Biology and Medicine,Basic Biological Research,Biochemistry,4,1.1442006,6,1.7163009,18,5.1489028,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,6,1.7163009,2013,9,2.5744514,NA,1,0.2860502,0,0.0000000,NA,33,9.439655,25,7.151254,8,2.2884013,216,27.00000,NA


In [150]:
# Shorten Preventive Medicine (Epidemiology/Early Diagnosis/Prevention)
grant_data <- mutate(grant_data, MainDiscipline_Level2 = ifelse(MainDiscipline_Level2 == "Preventive Medicine (Epidemiology/Early Diagnosis/Prevention)", "Preventive Medicine", MainDiscipline_Level2))

In [151]:
# Create factors
cols <- c(
    "State",
    "FundingInstrumentLevel1",
    "ResearchInstitution",
    "InstituteCountry",
    "MainDiscipline_Level1", 
    "MainDiscipline_Level2", 
    "MainDiscipline"
)

grant_data[, cols] <- lapply(grant_data[, cols], as.factor)

In [152]:
summary(grant_data)

  GrantNumber     CallDecisionYear EffectiveGrantStartDate GrantDurationDays
 Min.   :138726   Min.   :2012     Min.   :2012-02-01      Min.   :  27.0   
 1st Qu.:155115   1st Qu.:2014     1st Qu.:2014-10-01      1st Qu.: 122.0   
 Median :170810   Median :2016     Median :2017-04-01      Median : 548.0   
 Mean   :172289   Mean   :2016     Mean   :2017-03-18      Mean   : 701.8   
 3rd Qu.:190016   3rd Qu.:2019     3rd Qu.:2019-10-01      3rd Qu.:1095.0   
 Max.   :215930   Max.   :2022     Max.   :2023-02-01      Max.   :2524.0   
                                                                            
 GrantDurationYears EffectiveGrantEndDate       State     
 Min.   :0.07397    Min.   :2012-03-31    Completed:7609  
 1st Qu.:0.33425    1st Qu.:2017-03-31                    
 Median :1.50137    Median :2019-05-31                    
 Mean   :1.92282    Mean   :2019-02-18                    
 3rd Qu.:3.00000    3rd Qu.:2021-03-31                    
 Max.   :6.91507    Max.   :20

In [153]:
# Keep the columns of interest

grants <- select(grant_data,
  GrantNumber,
  CallDecisionYear,
  GrantDurationYears,
  FundingInstrumentLevel1,
  AmountGrantedAllSets,
  AmountPerYear,
  # ResearchInstitution,
  InstituteCountry,
  # MainDiscipline_Level1,
  MainDiscipline_Level2,
  # MainDiscipline,
  NrTotalUncitableOutputs,
  NrTotalUncitableOutputsPerYear,
  NrCitablePublications,
  NrCitablePublicationsPerYear,
  NrCitationsTotal,
  NrCitationsPerPublication)

In [154]:
# Keep the relevant medical grants only

grants <- filter(grants, 
                  MainDiscipline_Level2 %in% c(
                    "Basic Medical Sciences", 
                    "Experimental Medicine", 
                    "Clinical Medicine",
                    "Preventive Medicine",
                    "Social Medicine"
                    )
                )

In [155]:
dim(grants)
head(grants)

[1] 4115   14

GrantNumber,CallDecisionYear,GrantDurationYears,FundingInstrumentLevel1,AmountGrantedAllSets,AmountPerYear,InstituteCountry,MainDiscipline_Level2,NrTotalUncitableOutputs,NrTotalUncitableOutputsPerYear,NrCitablePublications,NrCitablePublicationsPerYear,NrCitationsTotal,NrCitationsPerPublication
<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
144100,2012,4.4958904,Projects,291191,64768.26,Switzerland,Preventive Medicine,44,9.786715,3,0.6672761,145,48.33333
145705,2012,0.9972603,Careers,66120,66301.65,United States of America,Basic Medical Sciences,0,0.000000,0,0.0000000,NA,NA
144172,2012,2.2493151,Projects,212782,94598.57,Switzerland,Clinical Medicine,0,0.000000,0,0.0000000,NA,NA
142663,2012,1.9972603,Careers,544935,272841.26,Switzerland,Basic Medical Sciences,4,2.002743,3,1.5020576,NA,NA
140514,2012,4.0000000,Projects,442000,110500.00,Switzerland,Basic Medical Sciences,0,0.000000,1,0.2500000,NA,NA
143748,2012,2.9972603,Projects,544800,181766.00,Switzerland,Basic Medical Sciences,8,2.669104,12,4.0036563,NA,NA


In [156]:
# Check for grants with at least 1 known publication and some known citations
test <- filter(grants, NrCitablePublications != 0)
test <- filter(grants, !is.na(NrCitationsTotal))

In [157]:
# Check the number of grants with 0 citation despite at least 1 known publication
sum(test$NrCitationsTotal == 0)

[1] 37

In [158]:
dim(test)

[1] 1960   14

In [159]:
# Keep only the grants which generated at least 1 publication with at least 1 citations as well as at least 1 general output
pub_grants <- filter(grants, NrCitablePublications != 0)
cited_grants <- filter(pub_grants, !is.na(NrCitationsTotal))
final_grants <- filter(cited_grants, NrCitationsTotal > 0)
# final_grants <- filter(cited_grants, NrTotalUncitableOutputs > 0)

In [160]:
dim(cited_grants)
head(cited_grants)

[1] 1923   14

GrantNumber,CallDecisionYear,GrantDurationYears,FundingInstrumentLevel1,AmountGrantedAllSets,AmountPerYear,InstituteCountry,MainDiscipline_Level2,NrTotalUncitableOutputs,NrTotalUncitableOutputsPerYear,NrCitablePublications,NrCitablePublicationsPerYear,NrCitationsTotal,NrCitationsPerPublication
<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
144100,2012,4.4958904,Projects,291191,64768.26,Switzerland,Preventive Medicine,44,9.786715,3,0.6672761,145,48.33333
139309,2012,4.9232877,Programmes,640027,129999.92,Switzerland,Social Medicine,13,2.640512,7,1.4218141,89,12.71429
145985,2012,0.9972603,Careers,42800,42917.58,Germany,Basic Medical Sciences,3,3.008242,1,1.0027473,146,146.00000
144072,2012,2.9972603,Projects,570301,190274.10,Switzerland,Experimental Medicine,1,0.333638,6,2.0018282,940,156.66667
142585,2012,2.9972603,Careers,593352,197964.79,Switzerland,Clinical Medicine,1,0.333638,10,3.3363803,242,24.20000
145760,2012,1.9972603,Careers,100000,50068.59,NA,Basic Medical Sciences,14,7.009602,4,2.0027435,214,53.50000


In [161]:
# Write  data to file
date_string <- format(Sys.Date(), "%Y%m%d")
file_name <- sprintf("BiomedGrants_%s.csv", date_string)

# Into tidy
write.table(final_grants, sprintf("4_%s", file_name), sep=";", fileEncoding="UTF-8", row.names = FALSE)

# Into analysis
write.table(final_grants, sprintf("../analysis/%s", file_name), sep=";", fileEncoding="UTF-8", row.names = FALSE)

# Into DaVi (comma-separated, not semi-colon)
write.table(final_grants, sprintf("../../visualization/%s", file_name), sep=",", fileEncoding="UTF-8", row.names = FALSE)